<a href="https://colab.research.google.com/github/fiocruzvig/abandono_tuberculose_SINAM/blob/main/TUB_AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Download das biblioteca do PySUS
!pip install pysus
!pip install --upgrade openpyxl

In [ ]:
#Importação das bibliotecas
import os
import pandas as pd
import numpy as np
from pysus.ftp.databases.sinan import SINAN
from openpyxl import load_workbook, Workbook

In [ ]:
#Carregando as informações do SINAN
sinan = SINAN().load()
files = sinan.get_files(dis_code="TUBE", year=[2022, 2021, 2020, 2019])
tube_2022, tube_2021 , tube_2020, tube_2019  = files

In [ ]:
sinan.describe(tube_2022)

{'name': 'TUBEBR19.dbc',
 'disease': 'Tuberculose',
 'year': 2019,
 'size': '5.2 MB',
 'last_update': '2024-04-22 04:23PM'}

In [ ]:
# Gerando a estruta de arquivos e obtendo os dados do SINAM
lista_tub = [tube_2022, tube_2021, tube_2020, tube_2019]
df = pd.DataFrame()
for tub in lista_tub:
   nome = str(tub)
   ano = nome[6:8]
   if not os.path.isfile(f'./TUB_CN/20{ano}/TUB_CN_20{ano}.csv'):
      os.makedirs(f"./TUB_CN/20{ano}")
      parquet = tub.download()
      df_new = parquet.to_dataframe()
      df_new.to_csv(f'./TUB_CN/20{ano}/TUB_CN_20{ano}.csv', index=False)
      df_temp = pd.read_csv(f'./TUB_CN/20{ano}/TUB_CN_20{ano}.csv', index_col=False, low_memory=False)
      df = pd.concat([df, df_temp])

#(378297, 97)
df.shape

(378297, 97)

In [ ]:
#Amostra
df = df[["CS_SEXO","NU_IDADE_N" ,"CS_RACA","CS_ESCOL_N" , "POP_LIBER", "POP_RUA","POP_SAUDE","POP_IMIG", "BENEF_GOV", "AGRAVDROGA", "AGRAVALCOO", "AGRAVTABAC" ,"AGRAVDIABE" ,"AGRAVDOENC", "AGRAVOUTRA","FORMA","HIV","TRATSUP_AT","SITUA_ENCE"]]
df.head()

,CS_SEXO,NU_IDADE_N,CS_RACA,CS_ESCOL_N,POP_LIBER,POP_RUA,POP_SAUDE,POP_IMIG,BENEF_GOV,AGRAVDROGA,AGRAVALCOO,AGRAVTABAC,AGRAVDIABE,AGRAVDOENC,AGRAVOUTRA,FORMA,HIV,TRATSUP_AT,SITUA_ENCE
0,F,4036.0,1.0,6.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,9.0,5.0
1,M,4025.0,4.0,7.0,2.0,2.0,2.0,2.0,9.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,NaN
2,M,4043.0,4.0,5.0,1.0,2.0,2.0,2.0,9.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,1.0,1.0,3.0
3,M,4027.0,4.0,1.0,2.0,2.0,2.0,2.0,9.0,NaN,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,5.0
4,F,4021.0,4.0,3.0,2.0,2.0,2.0,2.0,9.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,3.0


In [ ]:
# Cópia da Amostra
df_temp = df.copy()
df_temp = df_temp.reset_index(drop=True)



In [ ]:
### ABANDONO + ÓBITO + CURA ####
df_temp = df_temp[df_temp["SITUA_ENCE"].isin([1,2,3])]
df_temp = df_temp.reset_index(drop=True)
df_temp["SITUA_ENCE"].value_counts()


,count
SITUA_ENCE,
1.0,239600
2.0,56311
3.0,15134


In [ ]:
### ABANDONO + OUTROS(CURA & ÓBITO) ###
df_temp.replace({"SITUA_ENCE": {1: 0, 2: 1, 3: 0}}, inplace=True)
df_temp["SITUA_ENCE"].value_counts()


,count
SITUA_ENCE,
0.0,254734
1.0,56311


In [ ]:
# Função responsável por gerar o dataframe para cada variável explicativa
def funcao(df_ref , coluna, nome, df_aux=None):

    if df_aux is None:
      df_aux = df_temp.copy()

    df_aux = df_aux[df_aux["SITUA_ENCE"] == 1]
    df_aux =  df_aux[coluna].value_counts().reset_index(name="TOTAL_DE_ABANDONOS")
    df_ref = pd.merge(df_aux, df_ref.value_counts().reset_index(name="TOTAL"), on=coluna)
    df_ref["%_ABANDONOS"] = (df_ref['TOTAL_DE_ABANDONOS']/df_ref["TOTAL"])*100
    df_ref["%_ABANDONOS"] = df_ref["%_ABANDONOS"].round(2)
    df_ref.loc[-1] = [nome, df_ref["TOTAL_DE_ABANDONOS"].sum(), df_ref["TOTAL"].sum(), (df_ref["TOTAL_DE_ABANDONOS"].sum()/df_ref["TOTAL"].sum())*100]
    df_ref["%_ABANDONOS"] = df_ref["%_ABANDONOS"].round(2)
    df_ref.index = df_ref.index + 1
    df_ref.sort_index(inplace=True)
    return df_ref[[coluna, "TOTAL", "TOTAL_DE_ABANDONOS", "%_ABANDONOS"]]

In [ ]:
### SEXO ###
df_sexo = df_temp.copy()
df_sexo = df_sexo[["CS_SEXO"]]
df_sexo = funcao(df_sexo, "CS_SEXO", "Sexo", None)
df_sexo.rename(columns={"CS_SEXO": "SEXO"}, inplace=True)
df_sexo.replace({"M": "Masculino", "F": "Feminino", "I": "Ignorado"}, inplace=True)
df_sexo



,SEXO,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Sexo,311045,56311,18.10
1,Masculino,217595,43273,19.89
2,Feminino,93417,13031,13.95
3,Ignorado,33,7,21.21


In [ ]:
### Idade ###
df_idade = df_temp.copy()
df_idade["NU_IDADE_N"] = pd.cut(df_idade["NU_IDADE_N"],
                                  bins=[-np.inf, 4019, 4039, 4059, np.inf],
                                  labels=[1, 2, 3, 4])

if isinstance(df_idade["NU_IDADE_N"].dtype, pd.CategoricalDtype):
    current_categories = df_idade["NU_IDADE_N"].cat.categories
    if 0 not in current_categories:
        df_idade["NU_IDADE_N"] = df_idade["NU_IDADE_N"].cat.add_categories([0])



df_idade.fillna({"NU_IDADE_N": 0}, inplace=True)
categorias = {0: "Nulos", 1: "<=19 anos", 2: "20-39 anos", 3: "40-59 anos", 4: "60+ anos"}
df_idade["NU_IDADE_N"] = df_idade["NU_IDADE_N"].cat.rename_categories(categorias)

df_idade = funcao(df_idade[["NU_IDADE_N"]], "NU_IDADE_N", "Faixa_Etária", df_idade)
df_idade.rename(columns={"NU_IDADE_N": "FAIXA_ETARIA"}, inplace=True)
df_idade






,FAIXA_ETARIA,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Faixa_Etária,311045,56311,18.10
1,20-39 anos,145608,33193,22.80
2,40-59 anos,96173,15802,16.43
3,<=19 anos,24991,3745,14.99
4,60+ anos,44206,3564,8.06
5,Nulos,67,7,10.45


In [ ]:
### ESCOLARIDADE ###
df_esc = df_temp.copy()
df_esc["CS_ESCOL_N"] = pd.cut(df_esc["CS_ESCOL_N"], bins=[-np.inf, 0, 4,8,9,np.inf],
                                  labels=[1,2,3,4,5])

if isinstance(df_esc["CS_ESCOL_N"].dtype, pd.CategoricalDtype):
    current_categories =df_esc["CS_ESCOL_N"].cat.categories
    if 0 not in current_categories:
       df_esc["CS_ESCOL_N"] = df_esc["CS_ESCOL_N"].cat.add_categories([0])


df_esc.fillna({"CS_ESCOL_N": 0}, inplace=True)
categorias = {0: "Nulos", 1: "Sem_Escolaridade", 2: "<=8_anos", 3: ">8_anos", 4:"Ignorado", 5:"Não_aplicado"}
df_esc["CS_ESCOL_N"] = df_esc["CS_ESCOL_N"].cat.rename_categories(categorias)
df_esc = funcao(df_esc[["CS_ESCOL_N"]], "CS_ESCOL_N", "Escolaridade", df_esc)
df_esc.rename(columns={"CS_ESCOL_N": "ESCOLARIDADE"}, inplace=True)
df_esc


,ESCOLARIDADE,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Escolaridade,311045,56311,18.10
1,<=8_anos,115802,23630,20.41
2,>8_anos,93183,13525,14.51
3,Ignorado,64765,13152,20.31
4,Nulos,23888,4006,16.77
5,Sem_Escolaridade,9364,1404,14.99
6,Não_aplicado,4043,594,14.69


In [ ]:
### RAÇA ###
df_raca = df_temp.copy()
df_raca["CS_RACA"] = np.where(df_raca["CS_RACA"].between(2,5), 2, df_raca["CS_RACA"])
df_raca.fillna({"CS_RACA":0}, inplace=True)
df_raca.replace({"CS_RACA": {0: "Nulos", 1: "Branca", 2: "Não_Branca", 9:"Ignorado"}}, inplace=True)
df_raca = funcao(df_raca[["CS_RACA"]], "CS_RACA", "Raça", df_raca)
df_raca.rename(columns={"CS_RACA": "RAÇA"}, inplace=True)
df_raca

,RAÇA,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Raça,311045,56311,18.10
1,Não_Branca,203176,39169,19.28
2,Branca,85186,12665,14.87
3,Ignorado,15858,3318,20.92
4,Nulos,6825,1159,16.98


In [ ]:
### PRISIONEIROS ###
df_pri = df_temp.copy()
df_pri.replace({"POP_LIBER": {1: "Sim", 2: "Não",9:"Ignorado"}}, inplace=True)
df_pri.fillna({"POP_LIBER":"Nulos"}, inplace=True)
df_pri = funcao(df_pri[["POP_LIBER"]], "POP_LIBER", "Prisioneiro", df_pri)
df_pri.rename(columns={"POP_LIBER": "PRISIONEIRO"}, inplace=True)
df_pri

,PRISIONEIRO,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Prisioneiro,311045,56311,18.10
1,Não,265153,49054,18.50
2,Sim,35031,5272,15.05
3,Ignorado,5668,1159,20.45
4,Nulos,5193,826,15.91


In [ ]:
### MORADOR DE RUA ####
df_rua = df_temp.copy()
df_rua.replace({"POP_RUA": {1: "Sim", 2: "Não",9:"Ignorado"}}, inplace=True)
df_rua.fillna({"POP_RUA":"Nulos"}, inplace=True)
df_rua = funcao(df_rua[["POP_RUA"]], "POP_RUA", "Morador_de_Rua", df_rua)
df_rua.rename(columns={"POP_RUA": "MORADADOR_RUA"}, inplace=True)
df_rua

,MORADADOR_RUA,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Morador_de_Rua,311045,56311,18.10
1,Não,286202,48049,16.79
2,Sim,11150,5648,50.65
3,Ignorado,7713,1710,22.17
4,Nulos,5980,904,15.12


In [ ]:
### PROFISSIONAIS DA SAÚDE ###
df_saude = df_temp.copy()
df_saude.replace({"POP_SAUDE": {1: "Sim", 2: "Não",3:"Outros",9:"Ignorado"}}, inplace=True)
df_saude.fillna({"POP_SAUDE":"Nulos"}, inplace=True)
df_saude = funcao(df_saude[["POP_SAUDE"]], "POP_SAUDE", "Profissional_de_Saúde", df_saude)
df_saude.rename(columns={"POP_SAUDE": "PROFISSIONAL_SAUDE"}, inplace=True)
df_saude

,PROFISSIONAL_SAUDE,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Profissional_de_Saúde,311045,56311,18.10
1,Não,280987,51328,18.27
2,Outros,12021,2049,17.05
3,Ignorado,7661,1678,21.90
4,Nulos,6060,954,15.74
5,Sim,4316,302,7.00


In [ ]:
#### IMIGRANTE ####
df_imig = df_temp.copy()
df_imig.replace({"POP_IMIG": {1: "Sim", 2: "Não", 3:"Outros",9:"Ignorado"}}, inplace=True)
df_imig.fillna({"POP_IMIG":"Nulos"}, inplace=True)
df_imig = funcao(df_imig[["POP_IMIG"]], "POP_IMIG", "Imigrante", df_imig)
df_imig.rename(columns={"POP_IMIG": "IMIGRANTE"}, inplace=True)
df_imig

,IMIGRANTE,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Imigrante,311045,56311,18.10
1,Não,293278,52865,18.03
2,Ignorado,7978,1766,22.14
3,Nulos,6448,1030,15.97
4,Sim,1943,388,19.97
5,Outros,1398,262,18.74


In [ ]:
#### BENEFICIÁRIO DO GOVERNO ####
df_benef = df_temp.copy()
df_benef.replace({"BENEF_GOV": {1: "Sim", 2: "Não", 5: "Outros", 7: "Outros",3:"Outros", 4:"Outros", 8:"Outros",9:"Ignorado"}}, inplace=True)
df_benef.fillna({"BENEF_GOV":"Nulos"}, inplace=True)
df_benef = funcao(df_benef[["BENEF_GOV"]], "BENEF_GOV", "Beneficiário_Governo", df_benef)
df_benef.rename(columns={"BENEF_GOV": "BENEFICIARIO_GOV"}, inplace=True)
df_benef



,BENEFICIARIO_GOV,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Beneficiário_Governo,311039,56311,18.10
1,Não,166613,30200,18.13
2,Nulos,86865,15023,17.29
3,Ignorado,37136,7632,20.55
4,Sim,20425,3456,16.92


In [ ]:
### USUÁRIO DE DROGAS ###
df_drogas = df_temp.copy()
df_drogas.replace({"AGRAVDROGA": {1: "Sim", 2: "Não", 9:"Ignorado"}}, inplace=True)
df_drogas.fillna({"AGRAVDROGA":"Nulos"}, inplace=True)
df_drogas = funcao(df_drogas[["AGRAVDROGA"]], "AGRAVDROGA", "Drogas_Ilícitas", df_drogas)
df_drogas.rename(columns={"AGRAVDROGA": "DROGAS_ILICITAS"}, inplace=True)
df_drogas


,DROGAS_ILICITAS,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Drogas_Ilícitas,311045,56311,18.10
1,Não,239405,33642,14.05
2,Sim,50847,18223,35.84
3,Ignorado,16126,3683,22.84
4,Nulos,4667,763,16.35


In [ ]:
### ÁLCOOL ###
df_alcool = df_temp.copy()
df_alcool.replace({"AGRAVALCOO": {1: "Sim",2: "Não", 9:"Ignorado"}}, inplace=True)
df_alcool.fillna({"AGRAVALCOO":"Nulos"}, inplace=True)
df_alcool = funcao(df_alcool[["AGRAVALCOO"]], "AGRAVALCOO", "Álcool", df_alcool)
df_alcool.rename(columns={"AGRAVALCOO": "ALCOOL"}, inplace=True)
df_alcool

,ALCOOL,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Álcool,311045,56311,18.10
1,Não,234131,35662,15.23
2,Sim,58069,16491,28.40
3,Ignorado,15065,3530,23.43
4,Nulos,3780,628,16.61


In [ ]:
### TABACO ###
df_tabaco = df_temp.copy()
df_tabaco.replace({"AGRAVTABAC": {1: "Sim",2: "Não", 9:"Ignorado"}}, inplace=True)
df_tabaco.fillna({"AGRAVTABAC":"Nulos"}, inplace=True)
df_tabaco = funcao(df_tabaco[["AGRAVTABAC"]], "AGRAVTABAC", "Tabaco", df_tabaco)
df_tabaco.rename(columns={"AGRAVTABAC": "TABACO"}, inplace=True)
df_tabaco
#

,TABACO,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Tabaco,311045,56311,18.10
1,Não,213584,31895,14.93
2,Sim,77660,20037,25.80
3,Ignorado,15027,3547,23.60
4,Nulos,4774,832,17.43


In [ ]:
### DIABETES ###
df_diabetes = df_temp.copy()
df_diabetes.replace({"AGRAVDIABE": {1: "Sim",2: "Não", 9:"Ignorado"}}, inplace=True)
df_diabetes.fillna({"AGRAVDIABE":"Nulos"}, inplace=True)
df_diabetes = funcao(df_diabetes[["AGRAVDIABE"]], "AGRAVDIABE", "Diabetes", df_diabetes)
df_diabetes.rename(columns={"AGRAVDIABE": "DIABETES"}, inplace=True)
df_diabetes

,DIABETES,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Diabetes,311045,56311,18.10
1,Não,265866,49227,18.52
2,Ignorado,14764,3612,24.46
3,Sim,26431,2707,10.24
4,Nulos,3984,765,19.20


In [ ]:
### Outras Doenças ###
df_outras = df_temp.copy()
df_outras.replace({"AGRAVOUTRA": {1: "Sim",2: "Não", 9:"Ignorado"}}, inplace=True)
df_outras.fillna({"AGRAVOUTRA":"Nulos"}, inplace=True)
df_outras = funcao(df_outras[["AGRAVOUTRA"]], "AGRAVOUTRA", "Outras_Doenças", df_outras)
df_outras.rename(columns={"AGRAVOUTRA": "OUTRAS_DOENÇAS"}, inplace=True)
df_outras

,OUTRAS_DOENÇAS,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Outras_Doenças,311045,56311,18.10
1,Não,154023,27468,17.83
2,Nulos,108144,19641,18.16
3,Ignorado,26683,6121,22.94
4,Sim,22195,3081,13.88


In [ ]:
### PROBLEMAS MENTAIS ####
df_mental = df_temp.copy()
df_mental.replace({"AGRAVDOENC": {1: "Sim",2: "Não", 9:"Ignorado"}}, inplace=True)
df_mental.fillna({"AGRAVDOENC":"Nulos"}, inplace=True)
df_mental = funcao(df_mental[["AGRAVDOENC"]], "AGRAVDOENC", "Problemas_Mentais", df_mental)
df_mental.rename(columns={"AGRAVDOENC": "PROBLEMAS_MENTAIS"}, inplace=True)
df_mental

,PROBLEMAS_MENTAIS,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Problemas_Mentais,311045,56311,18.10
1,Não,283583,50181,17.70
2,Ignorado,15323,3669,23.94
3,Sim,7330,1549,21.13
4,Nulos,4809,912,18.96


In [ ]:
###  FORMAS ###
df_formas = df_temp.copy()
df_formas.replace({"FORMA": {1: "Pulmonar", 2: "Extrapulmonar", 3:"Pulmonar_Extrapulmonar"}}, inplace=True)
df_formas.fillna({"FORMA":"Nulos"}, inplace=True)
df_formas = funcao(df_formas[["FORMA"]], "FORMA", "Formas", df_formas)
df_formas.rename(columns={"FORMA": "FORMAS"}, inplace=True)
df_formas

,FORMAS,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Formas,311039,56311,18.10
1,Pulmonar,267408,50578,18.91
2,Extrapulmonar,35291,4201,11.90
3,Pulmonar_Extrapulmonar,8340,1532,18.37


In [ ]:
### HIV ###
df_hiv = df_temp.copy()
df_hiv.replace({"HIV": {1: "Sim", 2: "Não", 3:"Em_Andamento", 4:"Não_realizado"}}, inplace=True)
df_hiv.fillna({"HIV":"Nulos"}, inplace=True)
df_hiv = funcao(df_hiv[["HIV"]], "HIV", "HIV", df_hiv)
df_hiv


,HIV,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,HIV,311045,56311,18.10
1,Não,241754,38456,15.91
2,Não_realizado,42120,9466,22.47
3,Sim,24746,7825,31.62
4,Em_Andamento,1403,343,24.45
5,Nulos,1022,221,21.62


In [ ]:
### SUPERVISIONADO ####
df_sup = df_temp.copy()
df_sup.replace({"TRATSUP_AT": {1: "Sim",2: "Não",9:"Ignorado"}}, inplace=True)
df_sup.fillna({"TRATSUP_AT":"Nulos"}, inplace=True)
df_sup = funcao(df_sup[["TRATSUP_AT"]], "TRATSUP_AT", "Supervisionado", df_sup)
df_sup

,TRATSUP_AT,TOTAL,TOTAL_DE_ABANDONOS,%_ABANDONOS
0,Supervisionado,311045,56311,18.10
1,Não,124749,27746,22.24
2,Nulos,56019,13177,23.52
3,Sim,106325,10306,9.69
4,Ignorado,23952,5082,21.22


In [ ]:
dfs = []
dfs.append(df_sexo)
dfs.append(df_idade)
dfs.append(df_esc)
dfs.append(df_raca)
dfs.append(df_pri)
dfs.append(df_rua)
dfs.append(df_saude)
dfs.append(df_imig)
dfs.append(df_benef)
dfs.append(df_drogas)
dfs.append(df_alcool)
dfs.append(df_tabaco)
dfs.append(df_diabetes)
dfs.append(df_outras)
dfs.append(df_mental)
dfs.append(df_formas)
dfs.append(df_hiv)
dfs.append(df_sup)
len(dfs)

18

In [ ]:
### GERAR PLANILHAS ###
for d in dfs:
  d.to_excel(f"/content/{d.columns[0]}_AB.xlsx", index=False)

In [ ]:
import zipfile
import os
from google.colab import files

directory = '/content/'
zip_filename = f'/content/ar_xlsx.zip'
arquivos = []

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(directory):
        if root != directory:
          break
        for file in files:
            if file.endswith('.xlsx'):
                arquivos.append(os.path.join(root, file))
                zipf.write(os.path.join(root, file), arcname=file)

print(f"Arquivo ZIP criado: {zip_filename}")

Arquivo ZIP criado: /content/ar_xlsx.zip


In [ ]:
for arquivo in arquivos:
  if os.path.exists(arquivo):
    os.remove(arquivo)
    print(f"Diretório removido: {arquivo}")
  else:
     print(f"Diretório não removido: {arquivo}")

Diretório removido: /content/DROGAS_ILICITAS_AB.xlsx
Diretório removido: /content/IMIGRANTE_AB.xlsx
Diretório removido: /content/FAIXA_ETARIA_AB.xlsx
Diretório removido: /content/TOTAL_AB.xlsx
Diretório removido: /content/TRATSUP_AT_AB.xlsx
Diretório removido: /content/ESCOLARIDADE_AB.xlsx
Diretório removido: /content/FORMAS_AB.xlsx
Diretório removido: /content/PROFISSIONAL_SAUDE_AB.xlsx
Diretório removido: /content/SEXO_AB.xlsx
Diretório removido: /content/ALCOOL_AB.xlsx
Diretório removido: /content/BENEFICIARIO_GOV_AB.xlsx
Diretório removido: /content/DIABETES_AB.xlsx
Diretório removido: /content/OUTRAS_DOENÇAS_AB.xlsx
Diretório removido: /content/PROBLEMAS_MENTAIS_AB.xlsx
Diretório removido: /content/HIV_AB.xlsx
Diretório removido: /content/TABACO_AB.xlsx
Diretório removido: /content/PRISIONEIRO_AB.xlsx
Diretório removido: /content/MORADADOR_RUA_AB.xlsx
Diretório removido: /content/RAÇA_AB.xlsx
